<a href="https://colab.research.google.com/github/LLouzada/tcc/blob/master/nnunet_tcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Preparação para instalação nnUNet (Executar toda vez que reiniciar runtime)

from google.colab import drive
import os

# Diretório de trabalho
drive.mount('/content/drive',force_remount = False)
base_dir = '/content/drive/My Drive/Colab Notebooks'
os.chdir(base_dir)

Mounted at /content/drive


In [ ]:
### Clone do repositório nnUNet (if needed)

#!rm -rf nnUNet


#!git clone https://github.com/LLouzada/nnUNet

In [ ]:
### Instalação (Executar toda vez que reiniciar runtime)

respository_dir = os.path.join(base_dir,'nnUNet')
os.chdir(respository_dir)

!pip install -e .

!pip install --upgrade git+https://github.com/FabianIsensee/hiddenlayer.git

os.chdir(base_dir)


Obtaining file:///content/drive/My%20Drive/Colab%20Notebooks/nnUNet
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  Cloning https://github.com/FabianIsensee/hiddenlayer.git to /tmp/pip-req-build-oit13j6o
  Running command git clone --filter=blob:none --quiet https://github.com/FabianIsensee/hiddenlayer.git /tmp/pip-req-build-oit13j6o
  Resolved https://github.com/FabianIsensee/hiddenlayer.git to commit b7263b6dc4569da1b6dea5964e1eac78fa32fa77
  Preparing metadata (setup.py) ... done
  Created wheel for hiddenlayer: filename=hiddenlayer-0.2-py3-none-any.whl size=20004 sha256=ce84dbd14ba3fc4bede0a0318eb0d7a468a41cc780bda85b12122bcae75122bf
  Stored in directory: /tmp/pip-ephem-wheel-cache-k_9u5txf/wheels/55/0e/e3/fdf2f92789305c0e320e0ab01f27fd4b757b1bb01c07d532c4
Successfully built hiddenlayer


In [ ]:
### Unzip do dataset (if needed)
from google.colab import drive
import os
import zipfile


#drive.mount('/content/drive',force_remount = False)
base_dir = "/content/drive/My Drive/Colab Notebooks"
nnUNet_dataset_root_dir = os.path.join(base_dir, 'nnUNet_dataset')
nnUNet_dataset_raw_dir = os.path.join(nnUNet_dataset_root_dir, 'nnUNet_dataset_raw')
nnunet_raw_dir = os.path.join(nnUNet_dataset_raw_dir, 'nnUNet_raw')

os.chdir(nnUNet_dataset_root_dir)

# change the file if needed
zip_file = "nnUNet_raw_v3.zip"

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(nnUNet_dataset_raw_dir)

In [ ]:
# Run before running the cell below
def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input:
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder
    """
    if os.path.exists(folder_path):

        if not overwrite:
            print(f'{folder_path} exists.')
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
      os.makedirs(folder_path)
      print(f"{folder_path} created!")

In [ ]:
### nnUNet Setup

# Libraries
import shutil
from collections import OrderedDict
import json
import numpy as np
from google.colab import drive
import os
import matplotlib.pyplot as plt
import nibabel as nib
import zipfile

# Ambiente
drive.mount('/content/drive',force_remount = False)
base_dir = "/content/drive/My Drive/Colab Notebooks"
nnunet_dir = os.path.join(base_dir,'nnUNet')
nnUNet_dataset_root_dir = os.path.join(base_dir, 'nnUNet_dataset')
nnUNet_dataset_raw_dir = os.path.join(nnUNet_dataset_root_dir, 'nnUNet_dataset_raw')
nnunet_raw_dir = os.path.join(nnUNet_dataset_raw_dir, 'nnUNet_raw')
dataset_name = 'Dataset001_SacroRM'
dataset_id = "1"
dataset_dir = os.path.join(nnunet_raw_dir, dataset_name)
train_image_dir = os.path.join(dataset_dir,'imagesTr')
train_label_dir = os.path.join(dataset_dir,'labelsTr')
test_dir = os.path.join(dataset_dir,'imagesTs')
processed_dir = os.path.join(nnUNet_dataset_root_dir,'nnUNet_results')
result_dir = os.path.join(nnUNet_dataset_root_dir,'nnUNet_preprocessed')

# Checa se diretórios existem, caso conrario os cria
!echo "directories existence check:"
make_if_dont_exist(nnUNet_dataset_root_dir)
make_if_dont_exist(train_image_dir)
make_if_dont_exist(train_label_dir)
make_if_dont_exist(test_dir)
make_if_dont_exist(processed_dir)
make_if_dont_exist(result_dir)

# Env

os.environ['nnUNet_raw'] = nnunet_raw_dir
os.environ['nnUNet_preprocessed'] = processed_dir
os.environ['nnUNet_results'] = result_dir

# Checa se env está correto
!echo "environment variables check:"
!echo ${nnUNet_raw}
!echo ${nnUNet_preprocessed}
!echo ${nnUNet_results}

os.chdir(base_dir)

if os.getcwd()==base_dir:
    print('We are in the correct directory')
else:
    print("Run set base directory step again, then check to verify.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
directories existence check:
/content/drive/My Drive/Colab Notebooks/nnUNet_dataset exists.
/content/drive/My Drive/Colab Notebooks/nnUNet_dataset/nnUNet_dataset_raw/nnUNet_raw/Dataset001_SacroRM/imagesTr exists.
/content/drive/My Drive/Colab Notebooks/nnUNet_dataset/nnUNet_dataset_raw/nnUNet_raw/Dataset001_SacroRM/labelsTr exists.
/content/drive/My Drive/Colab Notebooks/nnUNet_dataset/nnUNet_dataset_raw/nnUNet_raw/Dataset001_SacroRM/imagesTs exists.
/content/drive/My Drive/Colab Notebooks/nnUNet_dataset/nnUNet_results exists.
/content/drive/My Drive/Colab Notebooks/nnUNet_dataset/nnUNet_preprocessed exists.
environment variables check:
/content/drive/My Drive/Colab Notebooks/nnUNet_dataset/nnUNet_dataset_raw/nnUNet_raw
/content/drive/My Drive/Colab Notebooks/nnUNet_dataset/nnUNet_results
/content/drive/My Drive/Colab Notebooks/nnUNet_dataset/nnUNet_preproces

In [ ]:

#colab users - mandatory


# os.chdir('apex')

# !pip install -v --no-cache-dir ./

# os.chdir(base_dir)


In [ ]:
!nnUNetv2_plan_and_preprocess -d 001 --verify_dataset_integrity

In [ ]:
# Fold 0
!nnUNetv2_train 001 2d 0 --c

In [ ]:
# Fold 1
!nnUNetv2_train 001 2d 1 --c

In [ ]:
# Fold 2
!nnUNetv2_train 001 2d 2 --c

In [ ]:
# Fold 3
!nnUNetv2_train 001 2d 3 --c

In [ ]:
# Fold 4
!nnUNetv2_train 001 2d 4 --c --npz

In [ ]:
# Getting the .npz for each fold (I forgot to add the arg --npz ;D )
!nnUNetv2_train 001 2d 0 --val --npz

In [ ]:
!nnUNetv2_train 001 2d 1 --val --npz

In [ ]:
!nnUNetv2_train 001 2d 2 --val --npz

In [ ]:
!nnUNetv2_train 001 2d 3 --val --npz

In [ ]:
!nnUNetv2_find_best_configuration 001 -c 2d

In [ ]:
!nnUNetv2_predict -d Dataset001_SacroRM -i '/content/drive/MyDrive/Colab Notebooks/nnUNet_dataset/nnUNet_dataset_raw/nnUNet_raw/Dataset001_SacroRM/imagesTs' -o '/content/drive/MyDrive/Colab Notebooks/nnUNet_dataset/nnUNet_prediction_results/Dataset001_SacroRM' -f  0 1 2 3 4 -tr nnUNetTrainer -c 2d -p nnUNetPlans

In [ ]:
# Result1
!nnUNetv2_predict -d Dataset001_SacroRM -i '/content/drive/MyDrive/Colab Notebooks/nnUNet_dataset/nnUNet_dataset_raw/nnUNet_raw/Dataset001_SacroRM/imagesTs' -o '/content/drive/MyDrive/Colab Notebooks/nnUNet_dataset/nnUNet_prediction_results/result1' -f  0 1 2 3 4 -tr nnUNetTrainer -c 2d -p nnUNetPlans

In [ ]:
# Result2
!nnUNetv2_predict -d Dataset001_SacroRM -i '/content/drive/MyDrive/Colab Notebooks/nnUNet_dataset/nnUNet_dataset_raw/nnUNet_raw/Dataset001_SacroRM/imagesTs' -o '/content/drive/MyDrive/Colab Notebooks/nnUNet_dataset/nnUNet_prediction_results/result2' -f  0 1 2 3 4 -tr nnUNetTrainer -c 2d -p nnUNetPlans

In [ ]:
# Result3
!nnUNetv2_predict -d Dataset001_SacroRM -i '/content/drive/MyDrive/Colab Notebooks/nnUNet_dataset/nnUNet_dataset_raw/nnUNet_raw/Dataset001_SacroRM/imagesTs' -o '/content/drive/MyDrive/Colab Notebooks/nnUNet_dataset/nnUNet_prediction_results/result3' -f  0 1 2 3 4 -tr nnUNetTrainer -c 2d -p nnUNetPlans


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 2 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 2 cases that I would like to predict

Predicting SRM_052:
perform_everything_on_gpu: True
100% 1/1 [00:08<00:00,  8.08s/it]
100% 1/1 [00:00<00:00, 32.66it/s]
100% 1/1 [00:00<00:00, 31.77it/s]
100% 1/1 [00:00<00:00, 31.12it/s]
100% 1/1 [00:00<00:00, 31.23it/s]
Prediction done, transferring to CPU if needed
sending off prediction to background worker for resampling and export
done with SRM_052

Predicting SRM_115:
perform_everything_on_gpu: True
100% 1/1 [00:00<00:00, 27.8

In [ ]:
# Post Processing
!nnUNetv2_apply_postprocessing -i "/content/drive/MyDrive/Colab Notebooks/nnUNet_dataset/nnUNet_prediction_results/result3" -o "/content/drive/MyDrive/Colab Notebooks/nnUNet_dataset/nnUNet_prediction_results/result3_pp" -pp_pkl_file "/content/drive/My Drive/Colab Notebooks/nnUNet_dataset/nnUNet_preprocessed/Dataset001_SacroRM/nnUNetTrainer__nnUNetPlans__2d/crossval_results_folds_0_1_2_3_4/postprocessing.pkl" -np 8 -plans_json "/content/drive/My Drive/Colab Notebooks/nnUNet_dataset/nnUNet_preprocessed/Dataset001_SacroRM/nnUNetTrainer__nnUNetPlans__2d/crossval_results_folds_0_1_2_3_4/plans.json"
